In [109]:
from mapping_reef_classes import get_coordinates
import os
import re
import pandas as pd
import numpy as np
import datetime
import pickle
import shutil

In [110]:
def list_files(filepath, filetype):
   paths = []
   for root, dirs, files in os.walk(filepath):
      for file in files:
         if file.lower().endswith(filetype.lower()):
            paths.append(os.path.join(root, file))
   return(paths)

In [29]:
tif_files = list_files(r"C:\Users\ageglio\OneDrive - DOI\Documents - Reef Mapping\Data Release", 'tif')
# further sorting of file search
tif_files = [re.findall('.*Bu.*', i) for i in tif_files]
# tif_files = [re.findall('.*Classified.*', i) for i in tif_files]
tif_files = list(set([item for sublist in tif_files for item in sublist]))
tif_files.sort()
tif_files

[]

In [15]:
lees_reef_tif = r"C:\Users\ageglio\Downloads\Lees Reef 1m.tif"
lees_reef = get_coordinates().return_min_max_tif_df([lees_reef_tif])
lees_reef

,filename,min_lat,min_lon,max_lat,max_lon,avr_lat,avr_lon,reef
0,Lees Reef 1m.tif,44.885416,-85.643245,44.980516,-85.598793,44.932966,-85.621019,Downloads


In [13]:
# 7, 8, 9
df_tiffs = get_coordinates().return_min_max_tif_df(tif_files)
# df_tiffs.to_csv("reef_coordinates.csv")
df_tiffs

,filename,min_lat,min_lon,max_lat,max_lon,avr_lat,avr_lon,reef
0,BayHarbor_Classified2.tif,45.370376,-85.000113,45.378667,-84.980203,45.374522,-84.990158,Bay Harbor
1,ElkRapids_Classified2.tif,44.895292,-85.429003,44.916075,-85.404111,44.905684,-85.416557,Elk Rapids
2,FishermensIsland_Classified.tif,45.286371,-85.372270,45.302994,-85.345161,45.294683,-85.358716,Fishermens Island
3,IngallsPoint_Classified.tif,45.067731,-85.577850,45.077120,-85.564201,45.072426,-85.571026,Ingalls Point
4,MissionPeninsula_Classified_test.tif,44.984333,-85.505051,45.035714,-85.469587,45.010023,-85.487319,Mission
5,TanneryCreek_Classified.tif,45.391005,-84.932999,45.409448,-84.917592,45.400226,-84.925296,Tannery Creek
6,TraverseShoal_Classified.tif,45.125140,-85.445749,45.152280,-85.420853,45.138710,-85.433301,Traverse Shoal


In [17]:
survey_names = pd.read_csv(r"Z:\HeadersCombined\combined\2019-2023_Collects_Survey_Names.csv", index_col=0)
df_collects = pd.read_csv("collects_lat_lon.csv", index_col=0).sort_values(by="date")
df_collects = pd.merge(df_collects, survey_names[["collect_id","# of bag files","SurveyName"]], on="collect_id", how="outer")
df_collects = df_collects[(df_collects["# of bag files"]>100)]
df_collects.to_csv("collects_lat_lon_survey_info.csv")

In [10]:
BayHarbor = df_tiffs.loc[0]
ElkRapids = df_tiffs.loc[1]
FishermansIsland = df_tiffs.loc[2]
IngallsPoint = df_tiffs.loc[3]
MissionPeninsula = df_tiffs.loc[4]
TanneryCreek = df_tiffs.loc[5]
TraverseShoal = df_tiffs.loc[6]

In [17]:
def reef_overlap(reef, df):
    df['Reef_overlap'] = np.nan
    apx = (1/60)/2
    idx = df[(reef.avr_lat > df.min_lat - apx) & (reef.avr_lat < df.max_lat + apx) & (reef.avr_lon > df.min_lon - apx) & (reef.avr_lon < df.max_lon + apx)].index
    new_df = df.loc[idx]
    new_df.loc[:,'Reef_overlap'] = reef.reef
    return new_df

In [12]:
BayHarbor_df = reef_overlap(BayHarbor, df_collects)
ElkRapids_df = reef_overlap(ElkRapids, df_collects)
FishermansIsland_df = reef_overlap(FishermansIsland, df_collects)
IngallsPoint_df = reef_overlap(IngallsPoint, df_collects)
MissionPeninsula_df = reef_overlap(MissionPeninsula, df_collects)
TanneryCreek_df = reef_overlap(TanneryCreek, df_collects)
TraverseShoal_df = reef_overlap(TraverseShoal, df_collects)
dfs = [BayHarbor_df, ElkRapids_df, FishermansIsland_df, IngallsPoint_df, MissionPeninsula_df, TanneryCreek_df, TraverseShoal_df]
reef_overlap_df = pd.concat(dfs)
reef_overlap_df = reef_overlap_df.sort_values(by=["Reef_overlap", "date"])
reef_overlap_df.to_csv("Reef_tif_collects_overlap.csv")

In [25]:
reef_overlap(lees_reef.loc[0], df_collects)

,collect_path,collect_id,date,collect,AUV,cam_sys,min_lat,min_lon,max_lat,max_lon,# of bag files,SurveyName,Reef_overlap


In [13]:
df_unp = pd.read_csv('all_unpacked_images_metadata.csv', index_col=0, low_memory=False)
df_unp_u = df_unp[~df_unp.Usability.isin(["Unusable","unusable"])]
## BayHarbor
imgs_BayHarbor = df_unp_u[df_unp_u.collect_id.isin(BayHarbor_df.collect_id.values)].index
## ElkRapids
imgs_ElkRapids = df_unp_u[df_unp_u.collect_id.isin(ElkRapids_df.collect_id.values)].index
## FishermansIsland
imgs_FishermansIsland = df_unp_u[df_unp_u.collect_id.isin(FishermansIsland_df.collect_id.values)].index
## IngallsPoint
imgs_IngallsPoint = df_unp_u[df_unp_u.collect_id.isin(IngallsPoint_df.collect_id.values)].index
## MissionPeninsula
imgs_MissionPeninsula = df_unp_u[df_unp_u.collect_id.isin(MissionPeninsula_df.collect_id.values)].index
## TanneryCreek
imgs_TanneryCreek = df_unp_u[df_unp_u.collect_id.isin(TanneryCreek_df.collect_id.values)].index
## TraverseShoal
imgs_TraverseShoal = df_unp_u[df_unp_u.collect_id.isin(TraverseShoal_df.collect_id.values)].index
imgs_BayHarbor

Index([213756, 213757, 213758, 213759, 213760, 213761, 213762, 213763, 213764,
       213765,
       ...
       215922, 215923, 215924, 215925, 215926, 215927, 215928, 215929, 215930,
       215931],
      dtype='int64', length=2135)

In [16]:
def copy_imgs_2_drive(df, group, dest_folder):
    header = df.loc[group]
    header_name = os.path.basename(dest_folder)+"_header.csv"
    header_dest = os.path.join(os.path.dirname(dest_folder), header_name)
    header.to_csv(header_dest)
    img_pth_list = list(df.loc[group, 'image_path'])
    img_nam_list = list(df.loc[group, 'image_name'])
    num_imgs = len(img_nam_list)
    i=0
    for img_pth, img_name in zip(img_pth_list, img_nam_list):
        src = img_pth
        dest = os.path.join(dest_folder, img_name)
        # File copy was interrupted often due to network, added src/dest comparison
        if os.path.exists(dest):
            if os.stat(src).st_size == os.stat(dest).st_size:
                i+=1
            else:
                shutil.copy(src, dest)
                i+=1
        else:
            shutil.copy(src, dest)
            i+=1
        print("Copying", i,"/",num_imgs, end='  \r')

dest = os.path.join("G:", "imgs_TraverseShoal")

## Stonecat
copy_imgs_2_drive(df_unp, imgs_TraverseShoal, dest_folder = dest)